In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import pathlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
import re
import shutil
import pandas as pd
import scipy.stats
import scipy
from functools import reduce
import exdir
import expipe
from distutils.dir_util import copy_tree
import septum_mec
import spatial_maps as sp
import head_direction.head as head
import septum_mec.analysis.data_processing as dp
import septum_mec.analysis.registration
from septum_mec.analysis.plotting import despine, savefig, plot_bootstrap_timeseries
from spatial_maps.fields import find_peaks, calculate_field_centers, separate_fields_by_laplace
from spike_statistics.core import permutation_resampling_test, block_bootstrap
import speed_cells.speed as spd
from tqdm import tqdm_notebook as tqdm
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

In [ ]:
project_path = dp.project_path()
project = expipe.get_project(project_path)
actions = project.actions

output_path = pathlib.Path("output") / "longitudinal-comparisons"
(output_path / "statistics").mkdir(exist_ok=True, parents=True)
(output_path / "figures").mkdir(exist_ok=True, parents=True)

# Load cell statistics and shuffling quantiles

In [ ]:
statistics_action = actions['calculate-statistics']
identification_action = actions['identify-neurons']
sessions = pd.read_csv(identification_action.data_path('sessions'))
units = pd.read_csv(identification_action.data_path('units'))
session_units = pd.merge(sessions, units, on='action')
statistics_results = pd.read_csv(statistics_action.data_path('results'))
statistics = pd.merge(session_units, statistics_results, how='left')
statistics.head()

In [ ]:
statistics['unit_day'] = statistics.apply(lambda x: str(x.unit_idnum) + '_' + x.action.split('-')[1], axis=1)

In [ ]:
stim_response_action = actions['stimulus-response']
stim_response_results = pd.read_csv(stim_response_action.data_path('results'))

In [ ]:
statistics = pd.merge(statistics, stim_response_results, how='left')

In [ ]:
print('N cells:',statistics.shape[0])

In [ ]:
shuffling = actions['shuffling']
quantiles_95 = pd.read_csv(shuffling.data_path('quantiles_95'))
quantiles_95.head()

In [ ]:
action_columns = ['action', 'channel_group', 'unit_name']
data = pd.merge(statistics, quantiles_95, on=action_columns, suffixes=("", "_threshold"))

data['specificity'] = np.log10(data['in_field_mean_rate'] / data['out_field_mean_rate'])

data.head()

# cell type

In [11]:
waveform_action = actions['waveform-analysis']
waveform_results = pd.read_csv(waveform_action.data_path('results')).drop('template', axis=1)

data = data.merge(waveform_results, how='left')

data.bs = data.bs.astype(bool)

data.loc[data.eval('t_i_peak == t_i_peak and not bs'), 'ns_inhibited'] = True
data.ns_inhibited.fillna(False, inplace=True)

data.loc[data.eval('t_i_peak != t_i_peak and not bs'), 'ns_not_inhibited'] = True
data.ns_not_inhibited.fillna(False, inplace=True)

data.loc[data.unit_id.isin(data.query('ns_inhibited').unit_id.values), 'ns_inhibited'] = True
data.loc[data.unit_id.isin(data.query('ns_not_inhibited').unit_id.values), 'ns_not_inhibited'] = True

## Find all cells with gridness above threshold

In [12]:
query = (
    'gridness > gridness_threshold and '
    'information_rate > information_rate_threshold and '
    'gridness > .2 and '
    'average_rate < 25'
)
sessions_above_threshold = data.query(query)
print("Number of sessions above threshold", len(sessions_above_threshold))
print("Number of animals", len(sessions_above_threshold.groupby(['entity'])))

Number of sessions above threshold 194
Number of animals 4


## select neurons that have been characterized as a grid cell on the same day

In [13]:
gridcell_sessions = data[data.unit_day.isin(sessions_above_threshold.unit_day.values)]
print("Number of gridcells", gridcell_sessions.unit_idnum.nunique())
print("Number of gridcell recordings", len(gridcell_sessions))
print("Number of animals", len(gridcell_sessions.groupby(['entity'])))

Number of gridcells 139
Number of gridcell recordings 230
Number of animals 4


In [14]:
data.loc[:,'gridcell'] = np.nan
data['gridcell'] = data.isin(gridcell_sessions)

data.loc[data.eval('not gridcell and bs'), 'bs_not_gridcell'] = True
data.bs_not_gridcell.fillna(False, inplace=True)

# Analysis

In [15]:
max_speed = .5 # m/s only used for speed score
min_speed = 0.02 # m/s only used for speed score
position_sampling_rate = 100 # for interpolation
position_low_pass_frequency = 6 # for low pass filtering of position

box_size = [1.0, 1.0]
bin_size = 0.02
smoothing_low = 0.03
smoothing_high = 0.06

speed_binsize = 0.02

stim_mask = True
# baseline_duration = 600
baseline_duration = None

In [16]:
data_loader = dp.Data(
    position_sampling_rate=position_sampling_rate, 
    position_low_pass_frequency=position_low_pass_frequency,
    box_size=box_size, bin_size=bin_size, 
    stim_mask=stim_mask, baseline_duration=baseline_duration
)

In [17]:
def fftcorrelate2d(arr1, arr2, normalize=False, **kwargs):
    from copy import copy
    arr1 = copy(arr1)
    arr2 = copy(arr2)
    from astropy.convolution import convolve_fft
    if normalize:
        # https://stackoverflow.com/questions/53436231/normalized-cross-correlation-in-python
        a_ = arr1.ravel()
        v_ = arr2.ravel()
        arr1 = (arr1 - np.mean(a_)) / (np.std(a_) * len(a_))
        arr2 = (arr2 - np.mean(v_)) / np.std(v_)
    corr = convolve_fft(arr1, np.fliplr(np.flipud(arr2)), normalize_kernel=False, **kwargs)
    return corr


def cross_correlation_distance(r1, r2):
    r12 = fftcorrelate2d(r1, r2)
    labels = separate_fields_by_laplace(r12, threshold=0)
    peaks = calculate_field_centers(r12, labels)
    centered_peaks = peaks - np.array(r1.shape) / 2
    offset = np.linalg.norm(centered_peaks, axis=1)
    distance_idx = np.argmin(offset)
    distance = offset[distance_idx]
    angle = np.arctan2(*centered_peaks[distance_idx])
    
    return distance, angle


def cross_correlation_centre_of_mass(r1, r2):
    from scipy import ndimage
    r12 = fftcorrelate2d(r1, r2)
    cntr = ndimage.center_of_mass(r12)
    
    centered_cntr = cntr - np.array(r1.shape) / 2
    distance = np.linalg.norm(centered_cntr)
    angle = np.arctan2(*centered_cntr)
    
    return distance, angle

In [18]:
colors = ['#1b9e77','#d95f02','#7570b3','#e7298a']
labels = [
    'Baseline Ia', 
    'Baseline Ib', 
    'Baseline I', 
    '11 Hz', 
    'Baseline IIa',
    'Baseline IIb',
    'Baseline II', 
    '30 Hz'
]
queries = [
    'baseline and i and Hz11',
    'baseline and i and Hz11',
    'baseline and i and Hz11',
    'frequency==11 and stim_location=="ms"', 
    'baseline and ii and Hz30',
    'baseline and ii and Hz30',
    'baseline and ii and Hz30',
    'frequency==30 and stim_location=="ms"']

In [19]:
keys = [
    'gridness',
    'speed_score',
    'max_rate',
    'average_rate',
    'xcorr_displacement',
    'xcorr_cntr_mass',
    'action',
    'unit_name'
]

cell_types = [
    'gridcell',
    'ns_inhibited', 
    'ns_not_inhibited',
    'bs',
    'bs_not_gridcell'
]

results_allcells = {}
results = {}
for cell_type in cell_types:
    results_allcells[cell_type] = {}
    results[cell_type] = {} # to store unique cells later
    for key in keys:
        results_allcells[cell_type][key] = list()
        for query, label in zip(queries, labels):
            values = data.query(query + f' and {cell_type}').loc[:,['entity', 'unit_idnum', 'channel_group', 'date', key]]
            results_allcells[cell_type][key].append(values.rename({key: label}, axis=1))
            
for cell_type, cell_type_val in results_allcells.items():
    for key, val in cell_type_val.items():
        df = reduce(lambda  left, right: pd.merge(left, right, on=['entity', 'unit_idnum', 'channel_group', 'date'], how='outer'), val)
        results_allcells[cell_type][key] = df#.drop('unit_day',axis=1)

/home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [20]:
results_allcells['gridcell']['action']

,entity,unit_idnum,channel_group,date,Baseline Ia,Baseline Ib,Baseline I,11 Hz,Baseline IIa,Baseline IIb,Baseline II,30 Hz
0,1833,30,0,260619,1833-260619-1,1833-260619-1,1833-260619-1,NaN,1833-260619-3,1833-260619-3,1833-260619-3,NaN
1,1833,31,0,260619,1833-260619-1,1833-260619-1,1833-260619-1,NaN,1833-260619-3,1833-260619-3,1833-260619-3,1833-260619-4
2,1833,32,0,260619,1833-260619-1,1833-260619-1,1833-260619-1,1833-260619-2,1833-260619-3,1833-260619-3,1833-260619-3,1833-260619-4
3,1833,78,1,260619,1833-260619-1,1833-260619-1,1833-260619-1,NaN,NaN,NaN,NaN,NaN
4,1833,79,1,260619,1833-260619-1,1833-260619-1,1833-260619-1,1833-260619-2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
145,1833,249,6,120619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1833-120619-4
146,1833,218,5,290519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1833-290519-4
147,1833,106,2,60619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1833-060619-2
148,1833,168,4,60619,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1833-060619-2


In [21]:
def drop_duplicates_least_null(df, key):
    return df.loc[df.notnull().sum(1).groupby(df[key]).idxmax()]

In [22]:
for cell_type, val in results_allcells.items():
    for key, df in val.items():
        results[cell_type][key] = drop_duplicates_least_null(df, 'unit_idnum')

In [23]:
results['gridcell']['action']

,entity,unit_idnum,channel_group,date,Baseline Ia,Baseline Ib,Baseline I,11 Hz,Baseline IIa,Baseline IIb,Baseline II,30 Hz
51,1833,8,0,20719,1833-020719-1,1833-020719-1,1833-020719-1,NaN,NaN,NaN,NaN,NaN
85,1833,13,0,20719,NaN,NaN,NaN,1833-020719-2,1833-020719-3,1833-020719-3,1833-020719-3,1833-020719-4
86,1833,14,0,20719,NaN,NaN,NaN,1833-020719-2,1833-020719-3,1833-020719-3,1833-020719-3,NaN
58,1833,23,0,200619,1833-200619-1,1833-200619-1,1833-200619-1,NaN,NaN,NaN,NaN,NaN
127,1833,26,0,200619,NaN,NaN,NaN,NaN,1833-200619-3,1833-200619-3,1833-200619-3,1833-200619-4
...,...,...,...,...,...,...,...,...,...,...,...,...
139,1849,835,4,150319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1849-150319-4
43,1849,851,5,60319,1849-060319-1,1849-060319-1,1849-060319-1,NaN,NaN,NaN,NaN,NaN
65,1849,932,7,280219,1849-280219-1,1849-280219-1,1849-280219-1,NaN,NaN,NaN,NaN,NaN
74,1849,937,7,280219,NaN,NaN,NaN,1849-280219-2,NaN,NaN,NaN,NaN


In [24]:
results['gridcell']['action'].groupby('entity').count()

,unit_idnum,channel_group,date,Baseline Ia,Baseline Ib,Baseline I,11 Hz,Baseline IIa,Baseline IIb,Baseline II,30 Hz
entity,,,,,,,,,,,
1833,94,94,94,43,43,43,38,36,36,36,24
1834,15,15,15,8,8,8,8,4,4,4,3
1839,20,20,20,9,9,9,6,5,5,5,6
1849,8,8,8,3,3,3,4,1,1,1,2


In [25]:
diff_labels = [
    ['Baseline Ia', 'Baseline Ib'], 
    ['Baseline I', '11 Hz'], 
    ['Baseline IIa', 'Baseline IIb'], 
    ['Baseline II', '30 Hz'],
    ['Baseline I', 'Baseline II']
]

## compute baseline values

In [26]:
def compute_baseline(column):
    total_itr = sum([results[cell_type]['action'][column].count() for cell_type in cell_types])
    pbar = tqdm(total=total_itr)

    for cell_type in cell_types:

        for idx, action_id in results[cell_type]['action'].loc[:,column].dropna().items():
            unit_name, channel_group = results[cell_type]['unit_name'].loc[idx, [column, 'channel_group']]
            rate_map_1, rate_map_2 = data_loader.rate_map_split(
                        action_id, channel_group, unit_name, smoothing_low)

            spikes_1, spikes_2, t_split = data_loader.spike_train_split(
                        action_id, channel_group, unit_name)

            t1, v1, t2, v2 = map(data_loader.tracking_split(action_id).get, ['t1', 'v1', 't2', 'v2'])

            pbar.update()
            for spikes, rate_map, speed, times, label in zip([spikes_1, spikes_2], [rate_map_1, rate_map_2], [v1, v2], [t1, t2], [column + 'a', column + 'b']):
                results[cell_type]['gridness'].loc[idx, label] = sp.gridness(rate_map)

                results[cell_type]['max_rate'].loc[idx, label] = rate_map.max()

                results[cell_type]['average_rate'].loc[idx, label] = len(spikes) / t_split

                results[cell_type]['speed_score'].loc[idx, label] = spd.speed_correlation(
                    speed, times, spikes, return_data=False)
    pbar.close()

In [27]:
compute_baseline('Baseline I')

/home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


/home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/elephant/statistics.py:835: UserWarning: Instantaneous firing rate approximation contains negative values, possibly caused due to machine precision errors.
  warnings.warn("Instantaneous firing rate approximation contains "
/home/mikkel/apps/expipe-project/septum-mec/septum_mec/analysis/data_processing.py:615: RuntimeWarning: divide by zero encountered in true_divide
  rate_map_1 = smooth_spike_map_1 / smooth_occupancy_map_1
/home/mikkel/apps/expipe-project/septum-mec/septum_mec/analysis/data_processing.py:615: RuntimeWarning: invalid value encountered in true_divide
  rate_map_1 = smooth_spike_map_1 / smooth_occupancy_map_1
/home/mikkel/apps/expipe-project/septum-mec/septum_mec/analysis/data_processing.py:616: RuntimeWarning: divide by zero encountered in true_divide
  rate_map_2 = smooth_spike_map_2 / smooth_occupancy_map_2
/home/mikkel/apps/expipe-project/septum-mec/septum_mec/analysis/data_processing.py:616: RuntimeWarnin

In [28]:
compute_baseline('Baseline II')

/home/mikkel/.virtualenvs/expipe/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
def compute_changing_statistics(results, labels, key):
    df = results[key]
    if key == 'gridness':
        values = df[labels[0]] - df[labels[1]]
    elif key == 'speed_score':
        values = df[labels[0]] - df[labels[1]]
    elif key == 'max_rate':
        values = (df[labels[0]] - df[labels[1]]) / df[labels[0]]
    elif key == 'average_rate':
        values = (df[labels[0]] - df[labels[1]]) / df[labels[0]]
    elif key == 'xcorr_cntr_mass':
        values = pd.Series(index=df.index)
        for idx, row in results['action'].loc[:, labels].dropna().iterrows():
            action_id_1, action_id_2 = row.loc[labels]
            unit_name_1, unit_name_2, channel_group = results['unit_name'].loc[idx, labels + ['channel_group']]

            rate_map_1 = data_loader.rate_map(
                action_id_1, channel_group, int(unit_name_1), smoothing_low)
            
            rate_map_2 = data_loader.rate_map(
                action_id_2, channel_group, int(unit_name_2), smoothing_low)
            distance, angle = cross_correlation_centre_of_mass(rate_map_1, rate_map_2)
            values[idx] = distance * bin_size
    elif key == 'xcorr_displacement':
        values = np.nan
    else:
        raise ValueError(key)
    
    return values

In [50]:
results_change = {}
for cell_type in cell_types:
    results_change[cell_type] = {}
    for key in keys:
        if key in ['action', 'unit_name']:
            continue
        results_change[cell_type][key] = pd.DataFrame()
        for diff_label in diff_labels:
            results_change[cell_type][key][' - '.join(diff_label)] = compute_changing_statistics(results[cell_type], diff_label, key)
            results_change[cell_type][key].loc[:, 'entity'] = results[cell_type][key].loc[:, 'entity']
            results_change[cell_type][key].loc[:, 'date'] = results[cell_type][key].loc[:, 'date']

In [51]:
results_change['gridcell']['gridness']

,Baseline Ia - Baseline Ib,entity,date,Baseline I - 11 Hz,Baseline IIa - Baseline IIb,Baseline II - 30 Hz,Baseline I - Baseline II
51,0.408873,1833,20719,NaN,NaN,NaN,NaN
85,NaN,1833,20719,NaN,0.076184,0.722833,NaN
86,NaN,1833,20719,NaN,0.132470,NaN,NaN
58,0.284086,1833,200619,NaN,NaN,NaN,NaN
127,NaN,1833,200619,NaN,0.077828,0.301516,NaN
...,...,...,...,...,...,...,...
139,NaN,1849,150319,NaN,NaN,NaN,NaN
43,0.147306,1849,60319,NaN,NaN,NaN,NaN
65,0.228274,1849,280219,NaN,NaN,NaN,NaN
74,NaN,1849,280219,NaN,NaN,NaN,NaN


# save to output folder

In [52]:
for cell_type, value_holder in results_change.items():
    for key, df in value_holder.items():
        path = output_path / "data" / "results_change" / cell_type
        path.mkdir(exist_ok=True, parents=True)
        df.to_csv(path / (key + '.csv'))
        
for cell_type, value_holder in results_allcells.items():
    for key, df in value_holder.items():
        path = output_path / "data" / "results_all_cells" / cell_type
        path.mkdir(exist_ok=True, parents=True)
        df.to_csv(path / (key + '.csv'))
        
for cell_type, value_holder in results.items():
    for key, df in value_holder.items():
        path = output_path / "data" / "results_unique_cells" / cell_type
        path.mkdir(exist_ok=True, parents=True)
        df.to_csv(path / (key + '.csv'))

# Plotting raw data

In [33]:
plt.rc('axes', titlesize=12)
plt.rcParams.update({
    'font.size': 12, 
    'figure.figsize': (6, 4), 
    'figure.dpi': 150
})

In [34]:
def make_plot(what, axs, action_ids, unit_names, channel_group):
    if what == 'rate_map':
        sns.despine(left=True, bottom=True)
        for ax, action_id, unit_name in zip(axs, action_ids, unit_names):
            if np.isnan(unit_name):
                continue
            vmax = None
            rate_map = data_loader.rate_map(action_id, channel_group, int(unit_name), smoothing_low)
            if vmax is None:
                vmax = rate_map.max()
            ax.imshow(rate_map, origin='lower', vmax=vmax)
    #             ax.set_title(f'{row.gridness:.2f} {row.max_rate:.2f} {row.average_rate:.2f}')
            ax.set_yticklabels([])
            ax.set_xticklabels([])
    if what == 'spike_map':
        from scipy.interpolate import interp1d
        for ax, action_id, unit_name in zip(axs, action_ids, unit_names):
            if np.isnan(unit_name):
                continue
            x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v'])
            ax.plot(x, y, 'k', alpha=0.3)
            spike_times = data_loader.spike_train(action_id, channel_group, unit_name)
            spike_times = spike_times[(spike_times > min(t)) & (spike_times < max(t))]
            x_spike = interp1d(t, x)(spike_times)
            y_spike = interp1d(t, y)(spike_times)
            ax.set_xticks([])
            ax.set_yticks([])
            ax.scatter(x_spike, y_spike, marker='.', color=(0.7, 0.2, 0.2), s=1.5)
            ax.set_yticklabels([])
            ax.set_xticklabels([])
            ax.set_aspect(1)
    if what == 'speed':
        colors = ['#1b9e77','#d95f02','#7570b3', '#e7298a']
        for ax, action_id, unit_name, color in zip(axs, action_ids, unit_names, colors):
            if np.isnan(unit_name):
                continue
            x, y, t, speed = map(data_loader.tracking(action_id).get, ['x', 'y', 't', 'v'])

            spike_times = data_loader.spike_train(action_id, channel_group, unit_name)
            spike_times = spike_times[(spike_times > min(t)) & (spike_times < max(t))]
        
            speed_score, inst_speed, rate, times = spd.speed_correlation(
                speed, t, spike_times, return_data=True)

            speed_bins = np.arange(min_speed, max_speed + speed_binsize, speed_binsize)
            ia = np.digitize(inst_speed, bins=speed_bins, right=True)
            rates = []

            for i in range(len(speed_bins)):
                rates.append(rate[ia==i])

            ax.set_title(f'{speed_score:.3f}')
            plot_bootstrap_timeseries(speed_bins, rates, ax=ax, normalize_values=True, color=color)
#             rr = [rr for r in rates for rr in r]
#             aspect = (max_speed - min_speed) / (np.nanmax(rr) - np.nanmin(rr))
            ax.set_aspect('auto')

In [35]:
pure_labels = [
    'Baseline I', 
    '11 Hz', 
    'Baseline II', 
    '30 Hz'
]

def plot(what, cell_type):

    n_neurons = len(results[cell_type]['action'])
    fig, axss = plt.subplots(n_neurons, 4, sharey=True, sharex=True, figsize=(8, 2*n_neurons))

    for (idx, row), axs in zip(results[cell_type]['action'].iterrows(), axss):
        action_ids = row.loc[pure_labels].values
        unit_names = results[cell_type]['unit_name'].loc[idx, pure_labels].values
        channel_group, unit_idnum, entity, date = row.loc[['channel_group', 'unit_idnum', 'entity', 'date']]
        axs[0].set_ylabel(f'{unit_idnum} {entity}-{date}')
        make_plot(what, axs, action_ids, unit_names, channel_group)
    plt.tight_layout()

    # fig.savefig(output_path / 'figures' / f'neuron_{id_num}_rate_map.png', bbox_inches='tight')
    # savefig(output_path / 'figures' / f'{what}-{cell_type}')
    fig.savefig(output_path / 'figures' / f'{what}-{cell_type}.svg', bbox_inches='tight')

In [36]:
%%capture
plot(cell_type='gridcell', what='rate_map')
plot(cell_type='gridcell', what='speed')
plot(cell_type='ns_inhibited', what='speed')

# Save to expipe

In [40]:
action = project.require_action("longitudinal-comparisons")

In [53]:
copy_tree(output_path, str(action.data_path()))

['/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/gridness.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/action.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/max_rate.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/xcorr_displacement.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/xcorr_cntr_mass.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/speed_score.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/average_rate.csv',
 '/media/storage/expipe/septum-mec/actions/longitudinal-comparisons/data/data/results_unique_cells/gridcell/unit_name

In [42]:
septum_mec.analysis.registration.store_notebook(action, "10_longitudinal_comparisons.ipynb")